# Sourced squeezed light feedback system

## Basic SLH Rules
Here are some of the basic rules of quantum input-output theory:

For a series product: 
$$
G_T(S_T,L_t,H_T) = G_2(S_2,L_2,H_2)  \triangleleft G_1(S_1,L_1,H_1) = \Bigg( S_1 S_2,L_2 + S_2 L_1, H_1 + H_2 + \frac{1}{2j}(L_2^\dagger S_2 L_1 - L_1^\dagger S_2^\dagger L_2) \Bigg)
$$

For a concatenation product (parallel connection):
$$
G_T(S_T,L_t,H_T) = G_1(S_1,L_1,H_1) \oplus G_2(S_2,L_2,H_2) = \Bigg( \begin{pmatrix} S_1 & 0 \\ 0 & S_2 \end{pmatrix}, \begin{bmatrix} L_1 \\ L_2 \end{bmatrix}, H_1 + H_2  \Bigg)
$$
Note: $\oplus$ is a subsitute for the concatenation product symbol.

For a direct coupling product (parallel conection with a direct couplnig connection): 
$$
G_T(S_T,L_t,H_T) = G_1(S_1,L_1,H_1) \oplus G_2(S_2,L_2,H_2) = \Bigg( \begin{pmatrix} S_1 & 0 \\ 0 & S_2 \end{pmatrix}, \begin{bmatrix} L_1 \\ L_2 \end{bmatrix}, H_1 + H_2 + H_{\text{int}} \Bigg)
$$
Note: $H_{\text{int}}$ is the interaction hamiltonian. 

The $\hbar$ is consider a natural unit ($\hbar = 1$). 


## Feedback Rules

A feedback rule is a special case of a series product. This rule specifies an output port x into inport y, which essentially reduces the component. First you make a concatenation product for respective unconnected components. This is what we'll define as G(S,L,H).
If the output of port x goes into input of port y, then the following procedure is followed:
$$
G_{x \rightarrow y}(S_{red},L_{red},H_{red}) = \Bigg( [S_{\bar{x} \bar{y}}+S_{\bar{x}y}(I-S_{xy})^{-1}S_{x\bar{y}}], [L_{\bar{x}}++S_{\bar{x}y}(I-S_{xy})^{-1}L_x], [H + \frac{1}{2i}(L^\dagger S_{:,y}(I-S_{xy})^{-1}L_x-L^\dagger_x(I-S^\dagger_{xy})^{-1}S^\dagger_{:,y}L] \Bigg)
$$
where
$S_{xy}$ is the (x,y) component of the S matrix, $S_{\bar{x}\bar{y}}$ is S without the xth row and yth column, $S_{:,y}$ is the entire yth column of S, $S_{\bar{x}y}$ is the yth column of S without the xth row,  and $S_{x\bar{y}}$ is the xth row of S without the yth column, and $L_{\bar{x}}$ is L without the xth row.

## Total triple SLH component

Here is the total connection of a two-level atom sourced feedback system of a Optical Parametric Oscillator and beamsplitter. The example is derived in the SLH in the overleaf document. 
$$
G_T(S_T,L_T,H_T) = \Bigg( 1, (l\sqrt{\kappa}a+\lambda(t)\sigma_-), j\epsilon (a^\dagger a^\dagger - aa) + \frac{l\sqrt{\kappa}}{2j} (a^\dagger \lambda(t) \sigma_- - \lambda(t)^*\sigma_- a) \Bigg),
$$
where
$$
l = \frac{\eta}{1+\sqrt{1-\eta^2}}; \lambda(t) = (\frac{\Omega^2}{2\pi})^{\frac{1}{4}}\frac{e^{\frac{-\Omega^2}{4}(t-t_c)^2}}{\sqrt{w(t)}}.
$$
w(t) = $\frac{1}{2}(erfc\{\frac{\sqrt{2}\Omega (t_c - t)}{2}\} + 1)$, $\eta$ is the transmission coefficient.

In [ ]:
# Import packages
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from qutip import *

In [ ]:
eta = 1/2
t_c = 0
kappa = (1)/(2*np.pi)
Omega = 0.5*kappa
epsilon = 1

l = eta/(1+np.sqrt(1-eta**2))

N_f = 25

sigma = Qobj([[0,1],[0,0]])

N_t = 10000
t_span = np.linspace(0,100,N_t)


In [ ]:
rho0 = tensor(basis(N_f,0),basis(2,1))
# Define bosonic operator
a = tensor(destroy(N_f), qeye(2))
adag = a.dag()
# Define atomic transition operator
sigma_m = tensor(qeye(N_f),sigma)
sigma_p = sigma_m.dag()

In [ ]:
def lamb(t,args):
    O = args['O']
    tc = args['tc']
    result = (O**2/(2*np.pi))**(1/4)*(np.exp(-(O**2/4)*(t-tc)**2))/(np.sqrt((1/2)*(sp.special.erfc((O*(tc-t))/np.sqrt(2))+1)))
    return result

In [ ]:
def lambc(t,args):
    O = args['O']
    tc = args['tc']
    lam = (O**2/(2*np.pi))**(1/4)*(np.exp(-(O**2/4)*(t-tc)**2))/(np.sqrt((1/2)*(sp.special.erfc((O*(tc-t))/np.sqrt(2))+1)))
    return lam.conj()
     

In [ ]:
L0 = l*np.sqrt(kappa)*a
L1 = sigma_m


In [ ]:
H0 = 1j*epsilon*(adag*adag-a*a)
H1 = ((l*np.sqrt(kappa))/(2j))*adag*sigma_m
H2 = -((l*np.sqrt(kappa))/(2j))*sigma_p*a

In [ ]:
H = [H0, [H1,lamb], [H2,lambc]]
Lops = [L0,[L1,lamb]]

In [ ]:
args = {'tc': t_c, 'O': Omega, 'll': l, 'k': kappa, 'b': a, 'bdag': adag, \
        'o_m': sigma_m, 'o_p': sigma_p}

In [ ]:
rho = mesolve(H, rho0,t_span,Lops, e_ops = [adag*a,sigma_p*sigma_m], args=args)

In [ ]:
num_phot = rho.expect[0]
atomic_state = rho.expect[1]
plt.figure(figsize = (10,10))
plt.plot(t_span,num_phot, label = 'Number of photons')
plt.plot(t_span,atomic_state, '-k',label = 'Atomic state')
plt.xlabel("time")
plt.ylabel("<$a^\dagger$a>, <$\sigma_-^\dagger \sigma_-$>")
plt.title('Squeezed Light and Atomic state')
#plt.axhline(0)
#plt.xlim(0,25)
#plt.ylim(0,1)
plt.legend()
plt.show()